In [14]:
import neo
from networkunit import tests, scores, models
#from networkunit.capabilities.cap_ProducesSpikeTrains import ProducesSpikeTrains
from networkunit.capabilities.ProducesSpikeTrains import ProducesSpikeTrains # branch v0.2
from copy import copy
import pandas as pd
import numpy as np
import elephant
import quantities as pq
import sciunit
from quantities import ms
import sys
sys.setrecursionlimit(10000)
from pathlib import Path

In [15]:
def create_class(path):
    """
    This function creates a model class for the specific file path given.
    """
    class model(models.loaded_spiketrains): 
    
        default_params = {'file_path':path}

        def load(self):
            file_path = self.default_params['file_path']        
            data = pd.read_csv(file_path,skiprows=2,sep='\t')
            spike_data = data.sort_values(by='time_ms')
            grouped = spike_data.groupby(spike_data['sender'])

            spike_trains = []
            for name, group in grouped:
                    """
                    Each group is senders and times for one value of senders. That is, we iterate through all 
                    neurons. And the times for each neuron is in sorted order. Therefore, the cvs
                    returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
                    """
                    t = np.asarray(group['time_ms'])
                    spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
                    spike_trains.append(spiketrain)
            return spike_trains
    return model

In [16]:
cont_seed_1 = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_continuous\\brunel_continuous_delay_1.0_2.0_seed_1_spikes_exc-12502-0.dat'
cont_1 = create_class(cont_seed_1)
# create instance of class
cont = cont_1()
# produce spike trains
cont.produce_spiketrains();

In [17]:
d2_seed_1 = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_1_2\\brunel_rounding_True_delay_1.0_2.0_seed_1_spikes_exc-12502-0.dat'
d2_1 = create_class(d2_seed_1)
# create instance of class
droop_1_2 = d2_1()
# produce spike trains
droop_1_2.produce_spiketrains();

In [18]:
len(cont.spiketrains)

10000

In [19]:
len(droop_1_2.spiketrains)

10000

In [20]:
class FR_test_class(sciunit.TestM2M, tests.firing_rate_test):
    score_type = scores.wasserstein_distance
    
FR_test = FR_test_class()

In [8]:
scores = FR_test.judge([cont,droop_1_2])

In [9]:
scores.score

,model,model
model,0.000000,1.317082
model,1.317082,0.000000
